In [8]:
import json

with open('documents.json','rt') as f:
    doc_raw = json.load(f)



In [6]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Step 2: Creating Embeddings using Pretrained models

In [10]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model = SentenceTransformer('all-mpnet-base-v2')

In [14]:
len(model.encode("This is a simple sentence"))

768

In [16]:
operations = []

for doc in documents:
    doc["text_vector"] = model.encode(doc['text']).tolist()
    operations.append(doc)

### Step 3: Setup Elastic Search Connection

In [17]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346161723137,
  -0.06891428679227829,
  -0.04448342323303223,
  0.006250153295695782,
  -0.042247094213962555,
  -0.00645399559289217,
  0.02754814364016056,
  -0.01861027255654335,
  -0.05203750357031822,
  -0.0024047393817454576,
  0.05745450034737587,
  -0.015592600218951702,
  0.015488838776946068,
  0.0987016037106514,
  0.01564955711364746,
  -0.061

In [27]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('https://reimagined-system-vw6pp66xrqxcwq56-9200.app.github.dev/')

es_client.info()

ApiError: ApiError(302, 'None')